In [20]:
import argparse
import warnings
import pandas as pd
import os
os.system("export PYTHONPATH=`readlink -f ./`")
from config import *
from houses import TEST_HOUSE,TRAIN_HOUSE
from archive.load_tepco import load_house_dataset_by_houses,load_house_dataset_by_houses_ex
from time2graph.utils.base_utils import Debugger
from time2graph.core.model_TEPCO import Time2Graph
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.under_sampling import RandomUnderSampler
from time2graph.core.shapelet_embedding import ShapeletEmbedding
from time2graph.core.shapelet_utils import transition_matrixs,__mat2edgelist,graph_embedding
testhouse = [str(i).zfill(3) for i in TEST_HOUSE]
trainhouse = [str(i).zfill(3) for i in TRAIN_HOUSE]

## 參數集合

In [2]:
behav='sleep'
class args:
    pass
args.seg_length,args.num_segment =5,5
args.cutpoints=[(0,3),(2,5)]
args.behav=behav
args.dataset = args.behav
args.K,args.C=20,40
args.opt_metric = 'accuracy'
args.init,args.warp=0,2
args.gpu_enable =True
args.tflag=True
args.embed,args.cmethod ='aggregate','greedy'
args.percentile =10
args.batch_size,args.embed_size =16,16
args.njobs=5
args.optimizer ,args.measurement='Adam','gdtw'
args.alpha,args.beta=0.1,0.05
args.scaled,args.norm=True,False
args.no_global = True
args.multi_graph,args.data_size =False,1
args.kernel,args.feature = 'dts','all'
args.n_splits = 5

In [3]:
if args.kernel=='xgb':
    opt_args= {
                'max_depth': 16,
                'learning_rate': 0.2,
                'scale_pos_weight': 1,
                'booster': 'gbtree'
            }
elif (args.kernel=='dts') or (args.kernel=='rf'):
    opt_args={
                'criterion': 'gini',
                'max_features': 'auto',
                'max_depth': 10,
                'min_samples_split': 4,
                'min_samples_leaf': 3,
                'class_weight': 'balanced'
            }
else:
    raise ValueError("please choose a classifier")

## load data

In [4]:
x_train, y_train, x_test, y_test,z_train,z_test = load_house_dataset_by_houses_ex(
        TEST_HOUSE=testhouse,TRAIN_HOUSE=trainhouse,assign_behavior=behav)
Debugger.info_print('data shape {}x{}'.format(x_train.shape[0],x_train.shape[1]))
Debugger.info_print('training: {:.2f} positive ratio with {}'.format(float(sum(y_train) / len(y_train)),
                                                                        len(y_train)))
Debugger.info_print('test: {:.2f} positive ratio with {}'.format(float(sum(y_test) / len(y_test)),
                                                                    len(y_test)))

[info]data shape 7296x25
[info]training: 0.34 positive ratio with 7296
[info]test: 0.37 positive ratio with 1824


In [5]:
m = Time2Graph(kernel=args.kernel, K=args.K, C=args.C, seg_length=args.seg_length,
                opt_metric=args.opt_metric, init=args.init, gpu_enable=args.gpu_enable,
                warp=args.warp, tflag=args.tflag, mode=args.embed,
                percentile=args.percentile, candidate_method=args.cmethod,
                batch_size=args.batch_size, njobs=args.njobs,
                optimizer=args.optimizer, alpha=args.alpha,
                beta=args.beta, measurement=args.measurement,
                representation_size=args.embed_size, data_size=args.data_size,
                scaled=args.scaled, norm=args.norm, global_flag=args.no_global,
                multi_graph=args.multi_graph,
                shapelets_cache='{}/scripts/cache/{}_{}_{}_{}_shapelets.cache'.format(
                    module_path, 
                    args.dataset, 
                    args.cmethod, args.K, args.seg_length),
                    feature_mode = args.feature,
                    label_all = args.behav,
                    cutpoints=args.cutpoints,
                )

[info]Time2GraphEmbed
[info]initialize t2g model with {'kernel': 'dts', 'kwargs': {'candidate_method': 'greedy', 'njobs': 5, 'optimizer': 'Adam', 'representation_size': 16, 'feature_mode': 'all', 'label_all': 'sleep', 'cutpoints': [(0, 3), (2, 5)]}, 'K': 20, 'C': 40, 'seg_length': 5, 'warp': 2, 'tflag': True, 'opt_metric': 'accuracy', 'mode': 'aggregate', 'batch_size': 16, 'gpu_enable': True, 'percentile': 10, 'shapelets': None, 'sembeds': None, 'clf': None, 'lr': 0.01, 'p': 2, 'alpha': 0.1, 'beta': 0.05, 'multi_graph': False, 'debug': True, 'measurement': 'gdtw', 'verbose': False, 'global_flag': True, 'cutpoints': [(0, 3), (2, 5)]}


In [6]:
cache_dir = '{}/scripts/cache/{}/'.format(module_path, args.dataset)
m.learn__shapelet(X=x_train, Y=y_train,Z=z_train, cache_dir=cache_dir)

[info]learn_shapelets


In [7]:
assert m.t2g.sembeds is None
for k in range(m.data_size):
    m.data_scaler[k].fit(x_train[:, :, k])
X_scaled = np.zeros(x_train.shape, dtype=np.float)
for k in range(m.data_size):
    X_scaled[:, :, k] = m.data_scaler[k].fit_transform(x_train[:, :, k])
X_scaled = np.zeros(x_train.shape, dtype=np.float)
if args.scaled:
    Debugger.info_print('scaled embedding model...')
    inputx=X_scaled
else:
    Debugger.info_print('unscaled embedding model...')
    inputx=x_train

[info]scaled embedding model...


In [ ]:
assert m.t2g.shapelets is not None, 'shapelets has not been learnt yet'
m.t2g.sembeds = ShapeletEmbedding(
    seg_length=args.seg_length, tflag=args.tflag, multi_graph=args.multi_graph,
    cache_dir=cache_dir, tanh=False, debug=m.t2g.debug,
    percentile=args.percentile, measurement=args.measurement, mode=args.embed,
    global_flag=args.no_global, 
    **m.t2g.kwargs)


In [19]:
# m.t2g.sembeds.fit(time_series_set=x_train[np.argwhere(y_train == 0).reshape(-1), :, :],
#                     shapelets=m.t2g.shapelets, warp=args.warp, init=args.init)
transition_set=transition_matrixs(
            time_series_set=x_train[np.argwhere(y_train == 0).reshape(-1), :, :], 
            shapelets=m.t2g.shapelets, seg_length=args.seg_length,
            tflag=args.tflag, multi_graph=args.multi_graph, tanh=False, debug=True,
            init=args.init, warp=args.warp, percentile=args.percentile, threshold=-1,
            measurement=args.measurement, global_flag=args.no_global,
            cutpoints=args.cutpoints
            )

[info]weight matrix between shapelet 0 15
[info](4803, 15, 1)
[info]threshold(10) 0.18452430814504625, mean 0.9767476916313171
[info]4803x3
[info]27030 edges involved in shapelets graph
[info]weight matrix between shapelet 10 25
[info](4803, 15, 1)
[info]threshold(10) 0.18448342978954319, mean 0.9765496253967285
[info]4803x3
[info]27199 edges involved in shapelets graph


In [ ]:
for idx,transition in enumerate(transition_set):
    tmat, sdist, dist_threshold = transition
    m.t2g.sembeds.dist_threshold = dist_threshold
    m.t2g.sembeds.embeddings.append(
        graph_embedding(
        tmat=tmat, num_shapelet=len(shapelets), embed_size=args.embed_size,
        cache_dir=args.cache_dir, **self.deepwalk_args)
    )

In [10]:
x = m.extract_features(X=x_train,Z=z_train, init=args.init,mode=args.feature)
max_accu, max_prec, max_recall, max_f1, max_metric = -1, -1, -1, -1, -1
metric_measure = m.return_metric_method(opt_metric=m.t2g.opt_metric)
m.train_classfit(x=x,Y=y_train,Z=z_train,n_splits=5,opt_args=opt_args)

[info]embedding threshold 0.18448342978954319
[info]sdist size (7296, 3, 20)
[info]sdist size (7296, 3, 20)
[info]using setup parameters executed by 100.00%
[info]classifier fit


In [11]:
y_pred = m.predict(X=x_test,Z=z_test)[0]

[info]embedding threshold 0.18448342978954319
[info]sdist size (1824, 3, 20)
[info]sdist size (1824, 3, 20)


In [12]:
Debugger.dc_print('{}\n{:.2f} positive ratio\nresult: accu {:.4f}, prec {:.4f}, recall {:.4f}, f1 {:.4f}'.format(
        args.cutpoints,float(sum(y_test) / len(y_test)),                                                                           
            accuracy_score(y_true=y_test, y_pred=y_pred),
            precision_score(y_true=y_test, y_pred=y_pred),
            recall_score(y_true=y_test, y_pred=y_pred),
            f1_score(y_true=y_test, y_pred=y_pred)
        ))

訊息已成功發送到Discord Webhook！
